# Safe dict reading

define a function `safe_dict(d, k)` that takes in a python dict `d` and a key `k` and makes it safe to read even with keys that aren't in the dictionary. If you try to read from the dictionary with a bad key, it should return 0 instead.

```
d = {1 : 2, 3 : 4}
safe_dict(d, 1) -> 2
safe_dict(d, 'cat') -> 0
```

In [71]:
def safe_dict(d,k):
    if k in d.keys():
        print(d[k])
    else:
        return 0
d = {"a": 2, "b" : 4}

safe_dict(d, "a")


2


# File Reading: Hamlet Exercises

Open `hamlet.txt` in the `data` folder

### 1. Mentionned Hamlet

How many times is hamlet mentioned in the book?

Use python and line iteration to count it up

In [16]:
with open("/Users/kalebmckenzie/Documents/GitHub/m1-4-files-strings/data/hamlet.txt", "r") as f:
    f_contents = f.read().lower()

ham_count = f_contents.count("hamlet")
print("Number of times Hamlet is repeated:", ham_count)


Number of times Hamlet is repeated: 474


### 2. File Reading as a .py program

Make a python file that defines a function that counts the number of times hamlet is mentionned using the code in the previous exercise.

Then import it in your notebook and call it here.

In [56]:
from ham import hamlet_c
print(hamlet_c())

474


### 3. Unique words in hamlet

Write a program that counts the unique words in hamlet.

In [8]:
import re
import string
frequency = {}
with open("/Users/kalebmckenzie/Documents/GitHub/m1-4-files-strings/data/hamlet.txt", "r") as f:
    text = f.read().lower()
    words = text.split()
    table = str.maketrans("", "", string.punctuation)
    stripped = [w.translate(table) for w in words]
    assemble = " ".join(stripped)
    matched_pattern = re.findall(r'\b[a-z]{1,30}\b', assemble)

for word in matched_pattern:
    count = frequency.get(word,0)
    frequency[word] = count + 1

frequency_list = frequency.keys()

print(len(frequency))


4588


# File Reading 2: A Python library.

In the `data` folder, you will find a folder called `csrgraph` which is a python library.

### 1. File count

Count the `py` files in the library using the `os` package

In [10]:
import os
files = os.listdir("/Users/kalebmckenzie/Documents/GitHub/m1-4-files-strings/data/csrgraph")
number_files = len(files)
print(number_files)

8


### 2. For the following packages, count the number of files that import them:

- pandas 

- numpy

- numba

In [15]:
import pandas
import os
pandas.__path__
panda = os.listdir("/Users/kalebmckenzie/Applications/anaconda3/lib/python3.7/site-packages/pandas")
panda_num = len(panda)
print(panda_num)



19


In [17]:
import numpy
import os
numpy.__path__
nump = os.listdir("/Users/kalebmckenzie/Applications/anaconda3/lib/python3.7/site-packages/numpy")
nump_num = len(nump)
print(nump_num)

27


In [19]:
import numba
import os
numba.__path__
numb = os.listdir("/Users/kalebmckenzie/Applications/anaconda3/lib/python3.7/site-packages/numba")
numb_num = len(numb)
print(numb_num)

119


# First NLP Program: IDF

Given a list of words, the the inverse document frequency (IDF) is a basic statistic of the amount of information of each word in the text.

The IDF formulat is:

$$IDF(w) = ln(\dfrac{N}{1 + n(w)})$$

Where:

- $w$ is the token (unique word),
- $n(w)$ is the number of documents that $w$ occurs in,
- $N$ is the total number of documents

Write a function, `idf(docs)` that takes in a list of lists of words and returns a dictionary  `word -> idf score`

Example:

```
IDF([['interview', 'questions'], ['interview', 'answers']]) -> {'questions': 0.0, 
                                                                'interview': -0.4, 
                                                                'answers': 0.0}


```

In [24]:
test_l = [['interview', 'questions'], ['interview', 'answers']]
docs = [item for elem in test_l for item in elem]
print(docs)


['interview', 'questions', 'interview', 'answers']


# Stretch Goal: TF-IDF on Hamlet

The TF-IDF score is a commonly used statistic for the importance of words. Its $\frac{TF}{IDF}$ where TF is the "term frequency" (eg. how often the words happens in the document).

Calculate the TF-IDF dictionary on the Hamlet book.

What's the TF-IDF of "Hamlet"?

What's the word with the highest TF-IDF in the book?

# Stretch Goal: Speaker count

Use a regular expression and looping over the `hamlet.txt` file to build a dictionary `character_name -> # times speaking`.

Who speaks the most often? Who speaks the least often?